In [1]:
# !pip3 install -U scikit-learn scipy matplotlib
# !pip install --upgrade rasterio
import numpy as np
import pandas as pd
import rasterio as rs

In [2]:
a_0 = f'a_0.tif'
a_1 = f'a_1.tif'
a_2 = f'a_2.tif'
a_3 = f'a_3.tif'

img0 = rs.open(a_0)
img1 = rs.open(a_1)
img2 = rs.open(a_2)
img3 = rs.open(a_3)

img0.shape
img0.meta

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': None,
 'width': 9609,
 'height': 4296,
 'count': 1,
 'crs': CRS.from_epsg(4326),
 'transform': Affine(8.983152841195215e-05, 0.0, 85.82407708642353,
        0.0, -8.983152841195215e-05, 25.57619883224871)}

In [3]:
data=pd.read_csv('BTP_a_0.csv')
data

,CLASS,Longitude,Latitude
0,2,86.519368,25.192734
1,2,86.543441,25.194933
2,2,86.499428,25.194989
3,1,86.170355,25.204459
4,1,86.175672,25.213510
...,...,...,...
240,1,86.521517,25.505425
241,1,86.509637,25.505931
242,1,86.285749,25.507130
243,1,86.294811,25.515322


In [4]:
import rasterio.transform
import rasterio as rio

# Create an Affine object
transform = rio.transform.Affine(8.983152841195215e-05, 0.0, 85.82407708642353,
                                 0.0, -8.983152841195215e-05, 25.57619883224871)

def add_xy_coordinates_and_classification(water):
    latitudes = water['Latitude']
    longitudes = water['Longitude']
    clf = water['CLASS']
    row, col = rasterio.transform.rowcol(transform, longitudes, latitudes)
    water["xc"] = np.array(row).astype(int)
    water["yc"] = np.array(col).astype(int)
    water["Classification"] = clf.astype(int)
    return water

In [5]:
add_xy_coordinates_and_classification(data)

,CLASS,Longitude,Latitude,xc,yc,Classification
0,2,86.519368,25.192734,4268,7739,2
1,2,86.543441,25.194933,4244,8007,2
2,2,86.499428,25.194989,4243,7517,2
3,1,86.170355,25.204459,4138,3854,1
4,1,86.175672,25.213510,4037,3913,1
...,...,...,...,...,...,...
240,1,86.521517,25.505425,787,7763,1
241,1,86.509637,25.505931,782,7631,1
242,1,86.285749,25.507130,768,5139,1
243,1,86.294811,25.515322,677,5240,1


In [6]:
# taking band values as numpy arrays
x0 = img0.read(1).astype(float)
x1 = img1.read(1).astype(float)
x2 = img2.read(1).astype(float)
x3 = img3.read(1).astype(float)

bands = [x0, x1, x2, x3]

classified_data = data[data['CLASS'].notnull()]

xc = classified_data['xc'].values.astype(int)
yc = classified_data['yc'].values.astype(int)

X = np.array([band[xc, yc] for band in bands]).T

data[['Band 1', 'Band 2', 'Band 3', 'Band 4']] = X
data.drop(["xc", "yc","Longitude","Latitude", "Classification" ],axis=1,inplace=True)
data

,CLASS,Band 1,Band 2,Band 3,Band 4
0,2,0.081659,0.014067,0.005508,0.006385
1,2,0.050733,0.003595,0.002719,0.003094
2,2,0.037993,0.001440,0.004863,0.008960
3,1,0.006182,0.001865,0.001548,0.001055
4,1,0.004793,0.000784,0.000051,0.000064
...,...,...,...,...,...
240,1,0.005246,0.000381,0.000698,0.000235
241,1,0.008409,0.002107,0.001634,0.000870
242,1,0.004117,0.000167,0.000322,0.000292
243,1,0.003673,0.000127,0.000517,0.000697


In [7]:
X = data.iloc[:, 1:5]
X
#input values  of all bands

,Band 1,Band 2,Band 3,Band 4
0,0.081659,0.014067,0.005508,0.006385
1,0.050733,0.003595,0.002719,0.003094
2,0.037993,0.001440,0.004863,0.008960
3,0.006182,0.001865,0.001548,0.001055
4,0.004793,0.000784,0.000051,0.000064
...,...,...,...,...
240,0.005246,0.000381,0.000698,0.000235
241,0.008409,0.002107,0.001634,0.000870
242,0.004117,0.000167,0.000322,0.000292
243,0.003673,0.000127,0.000517,0.000697


In [8]:
from sklearn.preprocessing import LabelEncoder
y = data.iloc[:,0]
le = LabelEncoder()
y = le.fit_transform(y)
y
#output label that indicate class 

array([1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 1, 1, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1, 0,
       2, 2, 0, 2, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3,
       2, 3, 2, 3, 3, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 3, 2,
       2, 4, 3, 3, 2, 2, 2, 4, 3, 3, 4, 3, 0, 3, 4, 4, 4, 4, 4, 4, 0, 4,
       4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 0, 0, 4, 0, 4, 4, 4, 0, 0, 0,
       4, 4, 4, 0, 4, 4, 4, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 4, 0, 4, 0, 0,
       4, 4, 4, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBClassifier
import multiprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=5)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

classifier = XGBClassifier()

params = {
    "learning_rate": [0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 0.22, 0.24, 0.26, 0.28, 0.30],
    "max_depth": [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7]
}

rs_model = RandomizedSearchCV(classifier, param_distributions=params, n_iter=5, scoring='roc_auc', n_jobs=multiprocessing.cpu_count(), cv=5, verbose=3)

rs_model.fit(X_train, y_train)

y_pred = rs_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


grid = GridSearchCV(rs_model.best_estimator_, params, verbose=2)
grid.fit(X_train, y_train)


from sklearn.model_selection import cross_val_score
score = cross_val_score(rs_model.best_estimator_, X, y, cv=10)
print("Accuracy: %.2f%%" % (score.mean() * 100))


In [10]:
X = np.array(bands).transpose((1, 2, 0)).reshape(-1, len(bands))
col_names = [f"Band {i}" for i, _ in enumerate(bands)]
df = pd.DataFrame(X, columns=col_names)

In [11]:
df

,Band 0,Band 1,Band 2,Band 3
0,0.021584,0.004604,0.005572,0.003688
1,0.023120,0.006421,0.004483,0.003188
2,0.024750,0.007500,0.003431,0.002109
3,0.026458,0.006954,0.004807,0.000596
4,0.027251,0.006697,0.007468,0.000782
...,...,...,...,...
41280259,0.029364,0.008565,0.004140,0.003823
41280260,0.029904,0.007874,0.006132,0.002231
41280261,0.030387,0.007376,0.007216,0.001795
41280262,0.030585,0.007989,0.007326,0.001988
